# Using ag-grid interactively in a jupyter notebook

This notebook features a jupyter widget enabling easy interaction with the very complex ag-grid library.
All the features of this lib are described on [ag-grid main documentation page](https://www.ag-grid.com/documentation-main/documentation.php).
You may see some features in this notebook, but we advise you to look at that doc for more complex information and features.

## Multioptions

In [1]:
import ipywidgets as widgets
from ipyaggrid import Grid
from ipyaggrid import get_license
from copy import deepcopy as copy
import os
import numpy as np
import pandas as pd

In [2]:
def mklbl(prefix, n):
    return ["%s%s" % (prefix, i) for i in range(n)]

miindex = pd.MultiIndex.from_product([mklbl('A', 4),
                                      mklbl('B', 2),
                                      mklbl('C', 4),
                                      mklbl('D', 2)],
                                    names=['RowIdx-1', 'RowIdx-2', 'RowIdx-3', 'RowIdx-4'])
index  =['-'.join(col).strip() for col in miindex.values]
micolumns = pd.MultiIndex.from_tuples([('a', 'foo', 'zap'),
                                       ('a', 'foo', 'zip'),
                                       ('a', 'bar', 'zap'),
                                       ('a', 'bar', 'zip'),
                                       ('b', 'foo', 'zap'),
                                       ('b', 'foo', 'zep'),
                                       ('b', 'bah', 'zep'),
                                       ('b', 'bah', 'zyp'),
                                       ('b', 'bah', 'zap'),
                                      ],
                                      names=['ColIdx-{}'.format(i) for i in range(1, 4)])
cols  =['-'.join(col).strip() for col in micolumns.values]
data = np.arange(len(miindex) * len(micolumns), dtype=np.float).reshape((len(miindex),len(micolumns)))
data = data.tolist()


dfrc = pd.DataFrame(data, index=miindex, columns=micolumns).sort_index().sort_index(axis=1)

dfr  = pd.DataFrame(data, index=miindex, columns=cols).sort_index().sort_index(axis=1)
dfr.columns.name = 'UniqueCol'

dfc  = pd.DataFrame(data, index=index, columns=micolumns).sort_index().sort_index(axis=1)
dfc.index.name = 'UniqueRow'

df  = pd.DataFrame(data, index=index, columns=cols).sort_index()
df.index.name = 'UniqueRow'
df.columns.name = 'UniqueCol'

In [3]:
css_rules = """
.number-cell {
    text-align: left;
}
.nb-even {
    color: red;
}
.nb-odd {
    color: blue;
}
"""

ccf = "function(params) { return params.value % 2 == 0 ? 'nb-even' : 'nb-odd'; }"
column_defs_1 = [{'field': df.index.name}] + [{'field': c, 'cellClass': ccf} for c in df.columns[:5]]

ccf = None
column_defs_2 = [{'field': df.index.name}] + [{'field': c} for c in df.columns[:5]]

grid_options = {
    'columnDefs' : column_defs_1,
    'enableSorting': 'true',
    'enableFilter': 'true',
    'enableColResize': 'false',
    'enableRangeSelection': True,
}

grid_options_2 = {
    'columnDefs' : column_defs_2,
    'enableSorting': 'false',
    'enableFilter': 'false',
    'enableColResize': 'true',
    'enableRangeSelection':True,
}

ag = Grid(grid_data=df,
            #grid_options=grid_options_2,
            grid_options_multi=[('opt1', grid_options), ('opt2', grid_options_2)],
            css_rules=css_rules,
            quick_filter=False,
            export_csv=True,
            show_toggle_edit=True,
            paste_from_excel=True,
            export_excel=False,
            export_mode="buttons",
            index=True,
            theme='ag-theme-balham')
ag

Grid(columns_fit='size_to_fit', compress_data=True, css_rules_down=['.number-cell {text-align: left;}', '.nb-e…

In [4]:
ag.grid_data_out['cols']

KeyError: 'cols'

# Quickfilter

In [ ]:
css_rules="""
.gridDiv"""

rowDat = [
            { 'make': 'Toyota', 'model': 'Celica', 'price': 35000 },
            { 'make': 'Ford', 'model': 'Mondeo', 'price': 32000 },
            { 'make': 'Porsche', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche2', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche4', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche45', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche56', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche1', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche43', 'model': 'Boxter', 'price': 72000 },
            { 'make': 'Porsche', 'model': 'Boxter', 'price': 73000 },
            { 'make': 'Porsche', 'model': 'Boxter', 'price': 72000 },
        ]

columnDef = [
            { 'headerName': 'Make', 'field': 'make'},
            { 'headerName': 'Model', 'field': 'model' },
            { 'headerName': 'Price', 'field': 'price'},
        ]

gridOptions = {'columnDefs': columnDef,}

a = Grid(grid_options = gridOptions,
         grid_data = rowDat,
         show_toggle_edit=True)

In [ ]:
a

In [ ]:
a.grid_data_out['cols']

## Using custom JS functions

In [ ]:
import urllib.request, json 
with urllib.request.urlopen("https://raw.githubusercontent.com/ag-grid/ag-grid-docs/master/src/olympicWinners.json") as url:
    data = json.loads(url.read().decode())

In [ ]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'width': 120, 'rowGroup': 'true'},
    {'headerName': "Year", 'field': "year", 'width': 90, 'pivot': 'true'},
    {'headerName': "Sport", 'field': "sport", 'width': 110, 'rowGroup': 'true'},
    {'headerName': "Athlete", 'field': "athlete"},
    {
        'headerName':'Results',
        'children':[
            {'headerName': "Gold", 'field': "gold", 'width': 100, 'aggFunc': 'sum'},
            {'headerName': "Silver", 'field': "silver", 'width': 100, 'aggFunc': 'sum'},
            {'headerName': "Bronze", 'field': "bronze", 'width': 100, 'aggFunc': 'sum'},
        ]
    }
];

gridOptions = {
    'pivotMode': 'true',
    'enableColResize': 'true',
    'columnDefs': columnDefs,
    'enableRangeSelection':True,
    'enableFilter':'true',
    'defaultColDef':{
        'enableRowGroup':True,
    },
    'enableSorting':'true',
    'animateRows':'true',
};

buttons = [{'name':'Log Toto', 'action':'console.log("toto");'},
           {'name':'Log Titi', 'action':'console.log("titi");'}]

pivot = Grid(export_mode='buttons',
             export_csv=True,
             export_excel=True,
             quick_filter=True,
             theme='ag-theme-balham',
             compress_data=True,
             grid_options=gridOptions,
             grid_data=data,
             menu={'buttons':buttons},
             columns_fit="auto",
             user_params={'slider_value':50},
             paste_from_excel=True,
            hide_grid=False, show_toggle_delete=True)

# Setting a simple slider to coordinate its value with the highlight button
slider = widgets.IntSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def on_slider_change(change):
    pivot.user_params = {'slider_value': change.new}

slider.observe(on_slider_change, names='value')

In [ ]:
slider

In [ ]:
pivot

In [ ]:
pivot.grid_data_out['grid']

## More tests

In [ ]:
columnDefs = [
    {'headerName': "Country", 'field': "country", 'rowGroup':True, 'hide':True},
    {'headerName': "Sport", 'field': "sport", 'rowGroup':True, 'hide':True},
    {
        'headerName': "Results",
        'children': [
            {'headerName': "Total", 'field': 'total', 'columnGroupShow': 'closed', 'aggFunc':'sum'},
            {'headerName': "Gold", 'field': 'gold', 'columnGroupShow': 'open', 'aggFunc':'sum'},
            {'headerName': "Silver", 'field': 'silver', 'columnGroupShow': 'open', 'aggFunc':'sum'},
            {'headerName': "Bronze", 'field': 'bronze', 'columnGroupShow': 'open', 'aggFunc':'sum'}
        ]
    },
]


grid_options = {
    'columnDefs': columnDefs,
    'enableFilter': True,
    'enableSorting':True,
}

grid = Grid(grid_data = data,
            grid_options=grid_options, 
            export_mode="buttons",
            theme='ag-theme-balham')

In [ ]:
grid

In [ ]:
grid.export_to_df = False

In [ ]:
grid.grid_data_out['grid']